### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.3 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR10データセットの前処理を定義
pre_transform_cifar10 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR10データセットの読み込み
pre_train_dataset_cifar10 = datasets.CIFAR10(root='/content/data/', download=True, transform=pre_transform_cifar10)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar10 = 0.0
pre_std_cifar10 = 0.0
pre_total_samples_cifar10 = len(pre_train_dataset_cifar10)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar10:
    pre_image, _ = data
    pre_mean_cifar10 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar10 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar10 /= pre_total_samples_cifar10
pre_std_cifar10 /= pre_total_samples_cifar10

print("データセット全体の平均値: ", pre_mean_cifar10)
print("データセット全体の標準偏差: ", pre_std_cifar10)

100%|██████████| 170498071/170498071 [00:05<00:00, 29461755.44it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.4914, 0.4822, 0.4465])
データセット全体の標準偏差:  tensor([0.2023, 0.1994, 0.2010])


In [5]:
# 学習用のCIFAR10データセットの前処理を定義
train_transform_cifar10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])
# テスト用のCIFAR10データセットの前処理を定義
test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])

# 学習用のCIFAR10データセットの読み込み
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=True, transform=train_transform_cifar10, download=True)
# テスト用のCIFAR10データセットの読み込み
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=False, transform=test_transform_cifar10, download=True)

# 学習用のCIFAR10データローダーを作成
train_loader_cifar10 = torch.utils.data.DataLoader(dataset=train_dataset_cifar10, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR10データローダーを作成
test_loader_cifar10 = torch.utils.data.DataLoader(dataset=test_dataset_cifar10, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### モデルの実装

In [6]:
# 1×1の畳み込みを定義
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3の畳み込みを定義
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# Residual Blocksを定義
class BasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_channels, channels, stride)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(channels, channels)
        self.bn2 = nn.BatchNorm2d(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Heの初期化（正規分布）
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                # 重みを1に初期化
                nn.init.constant_(m.weight, 1)
                # バイアスを0に初期化
                nn.init.constant_(m.bias, 0)

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [9]:
# モデルを呼び出す関数を定義
def resnet18_10():
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=10)

In [10]:
# パラメータ数を計算する関数を定義
def num_params(model):
    return sum(x.numel() for x in model.parameters() if x.requires_grad)
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを設定
model = resnet18_10().to(device)
# モデルのパラメータ数を確認
print("ResNet18のパラメータ数: {}".format(num_params(model)))

ResNet18のパラメータ数: 11181642


### 学習と評価（10 epochs）

In [11]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 10
# warm-upするepoch数を設定
warmup_epochs = 1

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR10_10epochs_CLRS_restest.pth')

Epoch [1/10], Train Accuracy: 31.49 %, Loss: 1.6316
Epoch [1/10], Learning Rate: 0.0001
Epoch [2/10], Train Accuracy: 43.43 %, Loss: 1.4183
Epoch [2/10], Learning Rate: 0.001
Epoch [3/10], Train Accuracy: 47.814 %, Loss: 1.2730
Epoch [3/10], Learning Rate: 0.0009779754323328191
Epoch [4/10], Train Accuracy: 56.33 %, Loss: 1.1417
Epoch [4/10], Learning Rate: 0.0009140576474687263
Epoch [5/10], Train Accuracy: 61.782 %, Loss: 1.0779
Epoch [5/10], Learning Rate: 0.000814503363531613
Epoch [6/10], Train Accuracy: 65.054 %, Loss: 0.9370
Epoch [6/10], Learning Rate: 0.0006890576474687264
Epoch [7/10], Train Accuracy: 68.146 %, Loss: 0.9242
Epoch [7/10], Learning Rate: 0.00055
Epoch [8/10], Train Accuracy: 70.456 %, Loss: 0.8076
Epoch [8/10], Learning Rate: 0.0004109423525312737
Epoch [9/10], Train Accuracy: 72.688 %, Loss: 0.7201
Epoch [9/10], Learning Rate: 0.00028549663646838715
Epoch [10/10], Train Accuracy: 74.226 %, Loss: 0.7428
Epoch [10/10], Learning Rate: 0.00018594235253127368


In [12]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 74.58 %


In [13]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（25 epochs）

In [14]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 25
# warm-upするepoch数を設定
warmup_epochs = 3

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR10_25epochs_CLRS_restest.pth')

Epoch [1/25], Train Accuracy: 31.854 %, Loss: 1.6775
Epoch [1/25], Learning Rate: 0.0001
Epoch [2/25], Train Accuracy: 42.644 %, Loss: 1.4393
Epoch [2/25], Learning Rate: 0.00039999999999999996
Epoch [3/25], Train Accuracy: 47.16 %, Loss: 1.3104
Epoch [3/25], Learning Rate: 0.0007
Epoch [4/25], Train Accuracy: 53.488 %, Loss: 1.1144
Epoch [4/25], Learning Rate: 0.001
Epoch [5/25], Train Accuracy: 58.852 %, Loss: 0.9739
Epoch [5/25], Learning Rate: 0.000996451615591515
Epoch [6/25], Train Accuracy: 63.422 %, Loss: 0.9849
Epoch [6/25], Learning Rate: 0.000985862422507884
Epoch [7/25], Train Accuracy: 66.582 %, Loss: 0.9279
Epoch [7/25], Learning Rate: 0.0009683994186497131
Epoch [8/25], Train Accuracy: 69.154 %, Loss: 0.9404
Epoch [8/25], Learning Rate: 0.0009443380060197386
Epoch [9/25], Train Accuracy: 70.876 %, Loss: 0.8355
Epoch [9/25], Learning Rate: 0.0009140576474687263
Epoch [10/25], Train Accuracy: 72.714 %, Loss: 0.8471
Epoch [10/25], Learning Rate: 0.0008780358823396353
Epoch 

In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 80.8 %


In [16]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（50 epochs）

In [14]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR10_50epochs_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 32.032 %, Loss: 1.6488
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 43.176 %, Loss: 1.5201
Epoch [2/50], Learning Rate: 0.00028
Epoch [3/50], Train Accuracy: 48.698 %, Loss: 1.4118
Epoch [3/50], Learning Rate: 0.00045999999999999996
Epoch [4/50], Train Accuracy: 53.504 %, Loss: 1.2398
Epoch [4/50], Learning Rate: 0.0006399999999999999
Epoch [5/50], Train Accuracy: 57.492 %, Loss: 1.1531
Epoch [5/50], Learning Rate: 0.00082
Epoch [6/50], Train Accuracy: 61.132 %, Loss: 0.8977
Epoch [6/50], Learning Rate: 0.001
Epoch [7/50], Train Accuracy: 64.172 %, Loss: 1.0784
Epoch [7/50], Learning Rate: 0.0009991120277927223
Epoch [8/50], Train Accuracy: 67.058 %, Loss: 0.8826
Epoch [8/50], Learning Rate: 0.000996451615591515
Epoch [9/50], Train Accuracy: 69.424 %, Loss: 0.8289
Epoch [9/50], Learning Rate: 0.00099202926282791
Epoch [10/50], Train Accuracy: 71.176 %, Loss: 0.8500
Epoch [10/50], Learning Rate: 0.000985862422507884
Epoch [11/50], Train

In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.18 %


In [16]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（75 epochs）

In [17]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 75
# warm-upするepoch数を設定
warmup_epochs = 8

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR10_75epochs_CLRS_restest.pth')

Epoch [1/75], Train Accuracy: 31.954 %, Loss: 1.6603
Epoch [1/75], Learning Rate: 0.0001
Epoch [2/75], Train Accuracy: 43.244 %, Loss: 1.4889
Epoch [2/75], Learning Rate: 0.00021250000000000002
Epoch [3/75], Train Accuracy: 48.122 %, Loss: 1.2742
Epoch [3/75], Learning Rate: 0.000325
Epoch [4/75], Train Accuracy: 52.924 %, Loss: 1.1634
Epoch [4/75], Learning Rate: 0.0004375
Epoch [5/75], Train Accuracy: 56.95 %, Loss: 1.1654
Epoch [5/75], Learning Rate: 0.00055
Epoch [6/75], Train Accuracy: 60.198 %, Loss: 1.1163
Epoch [6/75], Learning Rate: 0.0006625
Epoch [7/75], Train Accuracy: 62.906 %, Loss: 0.9053
Epoch [7/75], Learning Rate: 0.0007750000000000001
Epoch [8/75], Train Accuracy: 65.746 %, Loss: 0.9467
Epoch [8/75], Learning Rate: 0.0008875
Epoch [9/75], Train Accuracy: 67.342 %, Loss: 0.9543
Epoch [9/75], Learning Rate: 0.001
Epoch [10/75], Train Accuracy: 69.108 %, Loss: 0.7244
Epoch [10/75], Learning Rate: 0.0009996052735444863
Epoch [11/75], Train Accuracy: 71.044 %, Loss: 0.774

In [18]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.61 %


In [19]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（100 epochs）

In [11]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_10().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR10_100epochs_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.644 %, Loss: 1.6263
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 43.466 %, Loss: 1.4489
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 48.302 %, Loss: 1.3311
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 52.512 %, Loss: 1.3009
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 56.704 %, Loss: 1.2289
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 59.19 %, Loss: 1.0599
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 62.164 %, Loss: 1.0554
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 64.478 %, Loss: 0.9659
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 66.634 %, Loss: 0.9182
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 68.174 %, Loss: 0.9092
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 69.6

In [12]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 84.28 %


In [13]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()